#### This notebook contains code to analyze data using results from clustering model. A diagrammatic representation of the model development and data analysis approach is shown below:

<img alt="Diagrammatic representation of the analysis approach" src="static/QuantAnalysisApproach.png" title = "Model Dev and Analysis Approach"/>

#### <br>  The <strong> <em> Develop Model </em> </strong>  part is covered in a separate notebook - <em> DevelopModel.ipynb </em>

#### <br>  The <strong> <em> Analyze Results </em> </strong>  part is covered in this notebook

#### <br> Packages used:
|No.|Package Name|Version #|
|---|------------|---------|
|1|numpy|1.17.0|
|2|pandas|1.1.3|
|3| plotly|4.14.3 |
|4| scikitlearn|0.23.2 |
|5| scipy|1.5.2|

In [1]:
import numpy as np
import pandas as pd

import plotly.express as px


## I.	What are the distinct personas of professionals surveyed by Stackoverflow, given the diversity of the roles and the corresponding goals and objectives both for the organization and the professionals?

#### We load the cleaned dataset (which also contains the groups created using a K means clustering algorithm), aggregate and visualize data to answer the above question

In [2]:
#Read us community df generated from cluster analysis
us_community_df = pd.read_csv('data/us_community_df.csv',index_col=0)
us_community_df.head()

,Respondent,MainBranch,Hobbyist,Age,Age1stCode,CompFreq,CompTotal,ConvertedComp,Country,CurrencyDesc,...,Ef_Num_DT_DevFrontEnd,Ef_Num_DT_DevEmbedApps,Ef_Num_DT_DevGameOrGraphics,Ef_Num_DT_Designer,Ef_Num_DT_Scientist,Ef_Num_DT_DevMob,Ef_Num_DT_DevQATest,Ef_Num_DT_DevDeskOrEntApps,Ef_Num_DT_AcadResearch,Ef_Cat_DeveloperSegment
7,8,I am a developer by profession,Yes,36.0,12,Yearly,116000.0,116000.0,United States,United States dollar,...,205273.655261,206650.780408,235207.992063,207681.71528,208198.658609,207637.436534,210975.050212,212536.966460,210756.520125,0
13,14,I am a developer by profession,Yes,27.0,13,Yearly,66000.0,66000.0,United States,United States dollar,...,215996.556045,206650.780408,207930.254047,207681.71528,208198.658609,207637.436534,210975.050212,212536.966460,210756.520125,0
16,17,I am a developer by profession,Yes,25.0,14,Yearly,79000.0,79000.0,United States,United States dollar,...,205273.655261,206650.780408,207930.254047,207681.71528,208198.658609,207637.436534,210975.050212,208157.004262,210756.520125,0
17,18,I am a developer by profession,Yes,32.0,12,Monthly,105000.0,1260000.0,United States,United States dollar,...,205273.655261,206650.780408,207930.254047,207681.71528,208198.658609,207637.436534,210975.050212,208157.004262,210756.520125,0
18,19,I am a developer by profession,No,24.0,15,Yearly,83400.0,83400.0,United States,United States dollar,...,205273.655261,206650.780408,207930.254047,207681.71528,208198.658609,207637.436534,210975.050212,208157.004262,210756.520125,0


In [3]:
def format_cluster_headers(mean_comp,input_df):
    '''
    INPUT
        mean_comp - Mean compensation level across clusters (excluding missing values and outliers)
        input_df  - Input dataframe containing model input and outputs i.e. predicted clusters
    OUTPUT
        clust_hdr_lst - list containing cluster headers indicating extent of compensation level over mean value 
    '''
    
    pct_above_mean_comp = input_df['mean']/mean_comp
    pct_above_std_comp = input_df['std']/std_comp
    
    comp_matrix = pct_above_mean_comp.to_frame()
    comp_matrix.columns = ['Percentage']
    comp_matrix = comp_matrix.T
    comp_matrix_values = comp_matrix.values
    i=0
    clust_hdr_lst=[]
    for i in range(len(comp_matrix.values[0])):
        hdr_value = '<b>Grp' + str(i) + ' - CompvsMean: ' + str(round(comp_matrix.values[0][i],2))
        clust_hdr_lst.append(hdr_value)
        i+=1
    
    return clust_hdr_lst


def group_data_for_viz(input_df,group_colname_lst,disp_colname=''):
    '''
    INPUT
        input_df           - Input dataframe containing data to be grouped
        group_colname_list - list containing the column names on which grouping needs to be done
        disp_colname       - display column name (in case only given column is required for 
                             visualization); default value is blank i.e. entire dataset 
                             resulting from groupby operation
    OUTPUT
        df_g               - Dataset resulting from groupby operation
    '''
    #Compute figures required to display the chart
    df_g = input_df.groupby(group_colname_lst).size().reset_index()
    df_g['percentage'] = input_df.groupby(group_colname_lst).size().groupby(level=0).apply(lambda x: 100 * x / float(x.sum())).values
    df_g.percentage = df_g.percentage.round(2)
    agg_colnames = ['Count','Percentage']
    group_data_colname_lst = group_colname_lst + agg_colnames
    df_g.columns = group_data_colname_lst
    if disp_colname == '':
        return df_g
    else:
        df_g = df_g[df_g[disp_colname]==1]
        return df_g

def create_matrix(row,devtype,output_df):
    '''
    INPUT
        row       - Input row containing data to be aggregated
        devtype   - Each value of the developer type categorical variable 
        output_df - Updated output_df dataframe with the data contained in the input row
    OUTPUT
        None
    '''
    #devtype=row[colname]
    group=row['Ef_Cat_DeveloperSegment']
    percentage=row['Percentage']
    output_df.loc[devtype, group]= percentage
    return 

def agg_data (input_df, clustwise_comp_df,catcol_prefix,output_df):
    '''
    INPUT 
        input_df          - Dataframe containing data to be aggregated 
        catcol_prefix     - Category column prefix to be aggegrated 
        clustwise_comp_df - Cluster/ group header list
    OUTPUT
        output_df     - Aggregated data by Clustered groups
    '''
    
    catcol_list = [col for col in input_df.columns if catcol_prefix in col]
    catcol_prefix_index_list = catcol_list
    catcol_prefix_index_list = [cp.replace(catcol_prefix, '') for cp in catcol_prefix_index_list] 

    #groups=input_df.Ef_Cat_DeveloperSegment.unique().tolist()
    output_df=pd.DataFrame(data=0, index=catcol_prefix_index_list, columns=input_df.Ef_Cat_DeveloperSegment.unique().tolist()).sort_index()

    catcol_prefix_catcol_list = [col for col in input_df.columns if catcol_prefix in col]
    i=0
    for col in catcol_prefix_catcol_list:
        groupcol_list = [col] + ['Ef_Cat_DeveloperSegment']
        #print("group col",groupcol_list)
        #print("col",col)
        df_g = group_data_for_viz(input_df,groupcol_list,col)
        df_g.apply(lambda row : create_matrix(row,catcol_prefix_index_list[i],output_df), axis=1)
        i+=1

    newcolname_lst=[]
    for col in output_df.columns:
        newcolname = clustwise_comp_df[clustwise_comp_df.index == col].Ef_Cat_DeveloperSegmentDesc.values[0]
        newcolname_lst.append(newcolname)
    output_df.columns=newcolname_lst #rename columns

    output_df=output_df/100
    return output_df

def plot_heatmap(input_df,plot_title,plot_height,plot_width,html_filename,y=''):
    '''
    INPUT
        input_df - Dataframe containing the data to be plotted
        title    - plot title
        plot_height   - plot height 
        plot_width    - plot width
    
    OUTPUT
        None
    '''

    import plotly.express as px
    fig = px.imshow(input_df,
                    labels=dict(x="<b>DS Community Group<b>", y="<b>Roles<b>",color="<b>Percentage<b>"),title=plot_title,
                    x=input_df.columns,#['Group 0','Group 1','Group 2','Group 3'],
                    y=input_df.index,width=plot_width, height=plot_height
                   )
    fig.update_xaxes(side="bottom")
    if y == 'str':
        fig.update_yaxes(type='category')
    
    import plotly.io as pio
    pio.write_html(fig, file=html_filename, auto_open=True)
    
    fig.show()
    pass

def create_cont_matrix(row,agg_colname,output_df,mat_type='PCT'):
    '''
    INPUT
        row         - Data in each row for which contigency matrix needs to be created
        agg_colname - Data element i.e. Column name which constitutes the row in the contingency matrix
        output_df   - Contingency matrix dataset
        mat_type    - Indicator variable to indicate the column used for aggregating 
                      PCT - 'Percentage', 'RPCT' - Relative Percentage, or Count 
    OUTPUT
        None
    '''
    #devtype=row[colname]
    group=row['Ef_Cat_DeveloperSegment']
    if mat_type == 'PCT':
        value=row['Percentage']
    else:
        if mat_type == 'RPCT': #relative pct
            value=row['Relative_Pct']
        else:
            value=row['Count']
    #ind = int(row[agg_colname])
    output_df.loc[row[agg_colname], group]= value
    pass

## Functions to perform statistical tests 
def perform_chi_square_test(input_df,prob=0.95):
    '''
    INPUT
        input_df   - Input dataframe containing a contingency matrix of counts of combinations of 
                     pair of categorical variables to be tested for independence
        prob       - Probability level - default is 0.95; meaning alpha of 0.05
    OUTPUT
        None
    '''
    # chi-squared test with similar proportions
    from scipy.stats import chi2_contingency
    from scipy.stats import chi2

    cont_matrix_values = input_df.values
    stat, p, dof, expected = chi2_contingency(cont_matrix_values)
    print('dof=%d' % dof)
    print(expected)
    # interpret test-statistic
    prob = 0.95
    critical = chi2.ppf(prob, dof)
    print('probability=%.3f, critical=%.3f, stat=%.3f' % (prob, critical, stat))
    if abs(stat) >= critical:
        print('Result based on test statistic: Dependent (Reject null hypothesis H0)')
    else:
        print('Result based on test statistic: Independent (Fail to reject null hypothesis H0)')
    # interpret p-value
    alpha = 1.0 - prob
    print('significance=%.3f, p=%.3f' % (alpha, p))
    if p <= alpha:
        print('Result based on p-value: Dependent (Reject null hypothesis H0)')
    else:
        print('Result based on p-value: Independent (Fail to reject null hypothesis H0)')
    
    pass

In [4]:
# Determine the percentile rank of compensation level and create a categorical variable indicating
# the quartile in which the compensation level for each observation falls
us_community_df['Ef_PercentileConvComp'] = us_community_df.ConvertedComp.rank(pct = True)
us_community_df ['Ef_Cat_ConvertedCompQuartile'] = np.where(us_community_df['Ef_PercentileConvComp']<=0.25,1,
                                                   np.where((us_community_df['Ef_PercentileConvComp'] > 0.25) & (us_community_df['Ef_PercentileConvComp'] <= 0.5),2,
                                                   np.where(us_community_df['Ef_PercentileConvComp'] > 0.75,4,3)))

#Create a dataframe containing relative compensative levels of each group from mean and variability perspectives
country_mean =  us_community_df.ConvertedComp.mean()
country_std = us_community_df.ConvertedComp.std()
clustwise_comp_df = us_community_df.groupby(['Ef_Cat_DeveloperSegment']).describe() ['ConvertedComp'].loc[:,['mean','std']]
clustwise_comp_df ['Ef_Cat_DeveloperSegmentDesc'] = '<b>Grp' + clustwise_comp_df.index.astype(str) + ': Mean x: ' + round(clustwise_comp_df ['mean'] / country_mean,2).astype(str) + '; SD x :' + round(clustwise_comp_df ['std'] / country_std,2).astype(str)  
clustwise_comp_df ['relative_mean_multiple'] = round(clustwise_comp_df ['mean'] / country_mean,2)
clustwise_comp_df ['relative_sd_multiple'] = round(clustwise_comp_df ['std'] / country_std,2)
persona = ['DigiSolDev','DataProductDeploy','MidMgmt','DataProductDev','EmbedAppDev','AcadNDSRnD',
          'SeniorMgmt','TechOpsMgmt']
clustwise_comp_df ['persona'] = persona
clustwise_comp_df

,mean,std,Ef_Cat_DeveloperSegmentDesc,relative_mean_multiple,relative_sd_multiple,persona
Ef_Cat_DeveloperSegment,,,,,,
0,202379.349616,370638.739923,<b>Grp0: Mean x: 0.97; SD x :0.99,0.97,0.99,DigiSolDev
1,186367.700913,305363.827759,<b>Grp1: Mean x: 0.89; SD x :0.82,0.89,0.82,DataProductDeploy
2,247466.628906,426473.203576,<b>Grp2: Mean x: 1.18; SD x :1.14,1.18,1.14,MidMgmt
3,263221.188291,462685.640351,<b>Grp3: Mean x: 1.26; SD x :1.24,1.26,1.24,DataProductDev
4,220593.636015,387734.175737,<b>Grp4: Mean x: 1.05; SD x :1.04,1.05,1.04,EmbedAppDev
5,161594.786408,278170.223100,<b>Grp5: Mean x: 0.77; SD x :0.74,0.77,0.74,AcadNDSRnD
6,221591.712919,283339.088021,<b>Grp6: Mean x: 1.06; SD x :0.76,1.06,0.76,SeniorMgmt
7,224470.773463,351382.498172,<b>Grp7: Mean x: 1.07; SD x :0.94,1.07,0.94,TechOpsMgmt


In [6]:
#Aggregate Devtype by cluster group for visualization in the form of a Heatmap 
devtype_matrix=pd.DataFrame(data=[],columns=[])#Create an empty dataframe
devtype_matrix = agg_data (us_community_df, clustwise_comp_df,'Ef_Cat_DT_',devtype_matrix)
devtype_matrix.shape
plot_heatmap(devtype_matrix,'<b> Exhibit 2 - Role Types across groups',1000,1000,'static/Exhibit2.html')

## II.	How do these personas relate to compensation levels, both, practically and/or statistically?

In [7]:
import plotly.express as px
import plotly.graph_objects as go

fig = px.scatter(clustwise_comp_df, x="relative_mean_multiple", 
                     y="relative_sd_multiple", color="persona")
fig.add_trace(go.Scatter(x=[1.0,1,0], y=[0,1.4],
                    mode='lines',
                    name='Relative Mean Ratio = 1.0'))
fig.add_trace(go.Scatter(x=[0,1.4], y=[1.0,1,0],
                    mode='lines',
                    name='Relative SD Ratio = 1.0'))

fig.update_layout(
    title="<b>Exhibit 3 – Macro perspective: Relative comparison of compensation levels across groups",
    xaxis_title="<b>Relative Mean Ratio",
    yaxis_title="<b>Relative SD Ratio"
    )

fig.show()

In [8]:
#Aggregate compensation levels (categories i.e. quartile #) by cluster groups for visualization
df_g = group_data_for_viz(us_community_df,['Ef_Cat_DeveloperSegment' \
                                           ,'Ef_Cat_ConvertedCompQuartile'],disp_colname='')
cols=us_community_df.Ef_Cat_DeveloperSegment.unique().tolist()
cols.sort(key=int) #Sort the groups numerically in ascending order

#create df containing % membership in each of the 4 quartiles  for each of the  Cluster groups
compvsrolegroup_df =pd.DataFrame(data=0, index=us_community_df.Ef_Cat_ConvertedCompQuartile.unique().tolist(), columns=cols).sort_index()
x=df_g.apply(lambda row : create_cont_matrix(row,'Ef_Cat_ConvertedCompQuartile',compvsrolegroup_df), axis=1)
compvsrolegroup_df = compvsrolegroup_df/100
compvsrolegroup_df.columns=clustwise_comp_df.Ef_Cat_DeveloperSegmentDesc.tolist()
compvsrolegroup_df

,<b>Grp0: Mean x: 0.97; SD x :0.99,<b>Grp1: Mean x: 0.89; SD x :0.82,<b>Grp2: Mean x: 1.18; SD x :1.14,<b>Grp3: Mean x: 1.26; SD x :1.24,<b>Grp4: Mean x: 1.05; SD x :1.04,<b>Grp5: Mean x: 0.77; SD x :0.74,<b>Grp6: Mean x: 1.06; SD x :0.76,<b>Grp7: Mean x: 1.07; SD x :0.94
1,0.2715,0.1963,0.2188,0.2041,0.2069,0.4466,0.1483,0.1003
2,0.2711,0.2694,0.2070,0.2373,0.2605,0.2039,0.1340,0.1974
3,0.2382,0.2877,0.2773,0.2373,0.2299,0.1796,0.2057,0.3010
4,0.2192,0.2466,0.2969,0.3212,0.3027,0.1699,0.5120,0.4013


In [9]:
# Visualize information in the form of a Heatmap
fig = px.imshow(compvsrolegroup_df,
                    labels=dict(x="<b>DS Community Group<b>", y="<b>Compensation Level Quartiles<b>",color="<b>Percentage<b>"),
                    title= '<b>Exhibit 4 - Compensation Levels across groups<b>',
                    x=compvsrolegroup_df.columns,#['Group 0','Group 1','Group 2','Group 3'],
                    y=compvsrolegroup_df.index,width=700, height=500
                   )
fig.update_xaxes(side="bottom")
fig.update_yaxes(type='category')
fig.show()
#plot_heatmap(compvsrolegroup_df,'Compensation Levels across groups',800,1000,y='str')

In [10]:
#Get the compensation levels vs group dataframe as contingency matrix 
cols=us_community_df.Ef_Cat_DeveloperSegment.unique().tolist()
cols.sort(key=int) #Sort the groups numerically in ascending order

#create df containing contingency matrix to serve as input for chi square test
compvsrolegroup_contmat_df =pd.DataFrame(data=0, index=us_community_df.Ef_Cat_ConvertedCompQuartile.unique().tolist(), columns=cols).sort_index()
x=df_g.apply(lambda row : create_cont_matrix(row,'Ef_Cat_ConvertedCompQuartile',compvsrolegroup_contmat_df,'Count'), axis=1)
compvsrolegroup_contmat_df = compvsrolegroup_contmat_df.astype(int)
compvsrolegroup_contmat_df

,0,1,2,3,4,5,6,7
1,1484,86,56,129,108,92,31,31
2,1482,118,53,150,136,42,28,61
3,1302,126,71,150,120,37,43,93
4,1198,108,76,203,158,35,107,124


#### Conduct Statistical test (Chi square Test ) to check if compensation level (categorical variable) and cluster group categorical variable are independent

In [11]:
#Perform chi square test 
perform_chi_square_test(compvsrolegroup_contmat_df)

dof=21
[[1371.60014929  109.90868375   64.23886539  158.58969893  130.98706146
    51.69221199   52.4450112    77.53831799]
 [1407.64120428  112.7967156    65.92684747  162.7569047   134.42896243
    53.05051008   53.82309032   79.57576512]
 [1320.59865638  105.82184623   61.8502115   152.69270963  126.11644688
    49.77009206   50.49489923   74.65513809]
 [1366.15999005  109.47275442   63.98407564  157.96068674  130.46752924
    51.48718587   52.23699925   77.2307788 ]]
probability=0.950, critical=32.671, stat=267.248
Result based on test statistic: Dependent (Reject null hypothesis H0)
significance=0.050, p=0.000
Result based on p-value: Dependent (Reject null hypothesis H0)


#### Conclusion: Compensation levels quartile #  and Cluster group categorical variables are dependent

## III.	How do these personas relate to jobsat, both, practically and/or statistically?

In [12]:
print('Check number of null values for JobSat ',us_community_df.JobSat.isnull().value_counts(),'\n')
us_community_df.JobSat = np.where(us_community_df.JobSat.isnull(),"Not Specified",us_community_df.JobSat)
print('Value counts for JobSat \n')
us_community_df.JobSat.value_counts()

Check number of null values for JobSat  False    8030
True        8
Name: JobSat, dtype: int64 

Value counts for JobSat 



Very satisfied                        3312
Slightly satisfied                    2400
Slightly dissatisfied                 1134
Neither satisfied nor dissatisfied     693
Very dissatisfied                      491
Not Specified                            8
Name: JobSat, dtype: int64

In [13]:
#Calculate relative percentage diffeence between jobsat levels in group vis-a-vis corresponding 
#national percentage 
def calc_grp_vs_nat_rel_jobsat_levels(row,country_jobsat_pct):
    jobsat_level = row['Ef_Cat_JobSat']
    national_value_for_jobsat = country_jobsat_pct.loc[country_jobsat_pct.index==jobsat_level].values[0]
    relative_pct = row['Percentage'] - national_value_for_jobsat 
    return relative_pct

In [14]:
#Label encode JobSat values and aggregate data by cluster groups for visualization
from sklearn.preprocessing import LabelEncoder

# creating instance of labelencoder
labelencoder = LabelEncoder()
# Assigning numerical values and storing in another column
us_community_df['Ef_Cat_JobSat'] = labelencoder.fit_transform(us_community_df['JobSat'])
country_jobsat_pct = (us_community_df.Ef_Cat_JobSat.value_counts()/us_community_df.shape[0])*100
country_jobsat_pct

#Aggregate data 
df_g = group_data_for_viz(us_community_df,['Ef_Cat_DeveloperSegment','Ef_Cat_JobSat'],disp_colname='')
df_g ['Relative_Pct'] = df_g.apply(lambda row : \
                calc_grp_vs_nat_rel_jobsat_levels(row,country_jobsat_pct), axis=1)
df_g ['JobSat'] = labelencoder.inverse_transform(df_g.Ef_Cat_JobSat)

cols=us_community_df.Ef_Cat_DeveloperSegment.unique().tolist()
cols.sort(key=int) #Sort the groups numerically in ascending order
jobsatvsrolegroup_df =pd.DataFrame(data=0, index=us_community_df.Ef_Cat_JobSat.unique().tolist(), columns=cols).sort_index()
x=df_g.apply(lambda row : create_cont_matrix(row,'Ef_Cat_JobSat',jobsatvsrolegroup_df), axis=1)
jobsatvsrolegroup_df = jobsatvsrolegroup_df/100
jobsatvsrolegroup_df.columns=clustwise_comp_df.Ef_Cat_DeveloperSegmentDesc.tolist()
jobsatvsrolegroup_df.index = labelencoder.inverse_transform(jobsatvsrolegroup_df.index.values)
jobsatvsrolegroup_df

,<b>Grp0: Mean x: 0.97; SD x :0.99,<b>Grp1: Mean x: 0.89; SD x :0.82,<b>Grp2: Mean x: 1.18; SD x :1.14,<b>Grp3: Mean x: 1.26; SD x :1.24,<b>Grp4: Mean x: 1.05; SD x :1.04,<b>Grp5: Mean x: 0.77; SD x :0.74,<b>Grp6: Mean x: 1.06; SD x :0.76,<b>Grp7: Mean x: 1.07; SD x :0.94
Neither satisfied nor dissatisfied,0.0885,0.0822,0.0664,0.0712,0.0862,0.0922,0.0478,0.1197
Not Specified,0.0007,0.0000,0.0000,0.0016,0.0019,0.0049,0.0000,0.0032
Slightly dissatisfied,0.1473,0.1370,0.1641,0.1203,0.1360,0.1165,0.0957,0.1165
Slightly satisfied,0.2931,0.3607,0.3164,0.2896,0.3103,0.2718,0.2632,0.3333
Very dissatisfied,0.0598,0.0342,0.0664,0.0696,0.0747,0.0583,0.0766,0.0680
Very satisfied,0.4105,0.3858,0.3867,0.4478,0.3908,0.4563,0.5167,0.3592


In [15]:
def viz_relative_jobsat_levels(df,grp_header_list):
    '''
        INPUT
            df     - Input dataframe containing relative percentage of job satisfaction levels vis-a-vis
                     corresponding national percentages
            grp_header_list - List containing headers to be displayed for each cluster/ group
        OUTPUT
            fig    - Plotly graph object to be shown/ displayed
    '''
    import plotly.graph_objects as go
    from plotly.subplots import make_subplots
    
    #df = df1.loc[year_list,]
    fig = make_subplots(rows=3,cols=1, shared_xaxes=True, 
                   subplot_titles=('<b>JobSat - Slightly/ Very Satisfied'
                                   ,'<b>JobSat - Neutral/ Not Specified', 
                                    '<b>JobSat - Slightly/ Very Dissatisfied'))
    
    fig.add_trace(go.Bar(
        x=grp_header_list,
        y=df.loc[df['Ef_Cat_JobSat']==3].Relative_Pct,
        name = '<b> Slightly Satisfied <b>' # Style name/legend entry with html tags
        ), row = 1, col = 1)
    fig.add_trace(go.Bar(
        x=grp_header_list,
        y=df.loc[df['Ef_Cat_JobSat']==5].Relative_Pct,
        name = '<b> Very Satisfied <b>' # Style name/legend entry with html tags
        ), row = 1, col = 1)
    
    #plot Neutral and Notspecified jobsat categories in second row
    fig.add_trace(go.Bar(
        x=grp_header_list,
        y=df.loc[df['Ef_Cat_JobSat']==0].Relative_Pct,
        name = '<b> Neither Satisfied nor DIssatisfied <b>' # Style name/legend entry with html tags
        ), row = 2, col = 1)
    fig.add_trace(go.Bar(
        x=grp_header_list,
        y=df.loc[df['Ef_Cat_JobSat']==1].Relative_Pct,
        name = '<b> Not Specified <b>' # Style name/legend entry with html tags
        ), row = 2, col = 1)
    
    #Plot Dissatisfied job sat levels in the third row
    fig.add_trace(go.Bar(
        x=grp_header_list,
        y=df.loc[df['Ef_Cat_JobSat']==2].Relative_Pct,
        name = '<b> Slightly Dissatisfied <b>' # Style name/legend entry with html tags
        ), row = 3, col = 1)
    fig.add_trace(go.Bar(
        x=grp_header_list,
        y=df.loc[df['Ef_Cat_JobSat']==4].Relative_Pct,
        name = '<b> Very Dissatisfied <b>' # Style name/legend entry with html tags
        ), row = 3, col = 1)
    
    title_str = '<b> Exhibit 5 - Relative JobSat levels across groups'
    fig.update_layout(height=800, width=800,title=title_str),
    
    #fig.layout.template = 'plotly_dark'
    return fig

grp_header_list = clustwise_comp_df.Ef_Cat_DeveloperSegmentDesc.tolist()
fig = viz_relative_jobsat_levels(df_g,grp_header_list)
fig.show()

#### Conduct Statistical test (Chi square Test ) to check if JobSat level (categorical variable) and cluster group categorical variable are independent

In [16]:
#Get the compensation levels vs group dataframe as contingency matrix 
#df_g = group_data_for_viz(us_community_df,['Ef_Cat_DeveloperSegment','Ef_Cat_ConvertedCompQuartile'],disp_colname='')
cols=us_community_df.Ef_Cat_DeveloperSegment.unique().tolist()
cols.sort(key=int) #Sort the groups numerically in ascending order
jobsatvsrolegroup_contmat_df =pd.DataFrame(data=0, index=us_community_df.Ef_Cat_JobSat.unique().tolist(), columns=cols).sort_index()
x=df_g.apply(lambda row : create_cont_matrix(row,'Ef_Cat_JobSat',jobsatvsrolegroup_contmat_df,'Count'), axis=1)
jobsatvsrolegroup_contmat_df = jobsatvsrolegroup_contmat_df.astype(int)
jobsatvsrolegroup_contmat_df

#Perform chi sq test to test independence of two categorical variables - Cluster group & JobSat 
perform_chi_square_test(jobsatvsrolegroup_contmat_df)

dof=35
[[4.71253794e+02 3.77623787e+01 2.20711620e+01 5.44881811e+01
  4.50044787e+01 1.77603882e+01 1.80190346e+01 2.66405822e+01]
 [5.44015924e+00 4.35929336e-01 2.54789749e-01 6.29012192e-01
  5.19532222e-01 2.05026126e-01 2.08011943e-01 3.07539189e-01]
 [7.71142573e+02 6.17929833e+01 3.61164469e+01 8.91624782e+01
  7.36436925e+01 2.90624533e+01 2.94856930e+01 4.35936800e+01]
 [1.63204777e+03 1.30778801e+02 7.64369246e+01 1.88703658e+02
  1.55859667e+02 6.15078378e+01 6.24035830e+01 9.22617567e+01]
 [3.33889774e+02 2.67551630e+01 1.56377208e+01 3.86056233e+01
  3.18862901e+01 1.25834785e+01 1.27667330e+01 1.88752177e+01]
 [2.25222593e+03 1.80474745e+02 1.05482956e+02 2.60411048e+02
  2.15086340e+02 8.48808161e+01 8.61169445e+01 1.27321224e+02]]
probability=0.950, critical=49.802, stat=58.004
Result based on test statistic: Dependent (Reject null hypothesis H0)
significance=0.050, p=0.009
Result based on p-value: Dependent (Reject null hypothesis H0)


#### Conclusion: JobSat and Cluster group categorical variables are dependent